In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import glob
import seaborn as sns
import statsmodels.api as sm
import sklearn.preprocessing

In [2]:
# Read in rb stats files
wrStatsFiles = [filename for filename in glob.glob("C:/Users/Michael/Desktop/Springboard/Capstone2/csvFiles/DFS_Data/WR/stats/*.csv")]
wrStatsFiles[0]

'C:/Users/Michael/Desktop/Springboard/Capstone2/csvFiles/DFS_Data/WR/stats\\WR2011.csv'

In [3]:
# Extract season and fix playoff weeks to match other tables

dfs = []
dfsPlayoffs = []

for i in range(int(len(wrStatsFiles)/2)):
    df = pd.read_csv(wrStatsFiles[i])
    df['season'] = wrStatsFiles[i][-8:-4]
    dfs.append(df)

# second half of list has playoff games, need to fix their weeks, then combine with first table    
for i in range(int(len(wrStatsFiles)/2),int(len(wrStatsFiles)/2)*2):
    dfPlayoffs = pd.read_csv(wrStatsFiles[i])
    dfPlayoffs['season'] = wrStatsFiles[i][-8:-4]
    dfsPlayoffs.append(dfPlayoffs)

    
for i in range(len(dfsPlayoffs)):
    dfsPlayoffs[i].loc[dfsPlayoffs[i].Week == 1, 'Week'] = 18
    dfsPlayoffs[i].loc[dfsPlayoffs[i].Week == 2, 'Week'] = 19
    dfsPlayoffs[i].loc[dfsPlayoffs[i].Week == 3, 'Week'] = 20
    dfsPlayoffs[i].loc[dfsPlayoffs[i].Week == 4, 'Week'] = 21


# dfsPlayoffs[-1].head()

wr = pd.concat(dfs,axis=0)
wr = wr.append(dfsPlayoffs)
wr.head()

,Rank,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,...,ReceivingYardsPerReception,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameFanDuel,FantasyPointsFanDuel,season
0,1,Wes Welker,NE,WR,3,BUF,20,16,80.0,217,...,13.6,1,19,19.0,0,0,0,43.6,43.6,2011
1,2,Jordy Nelson,GB,WR,17,DET,15,9,60.0,162,...,18.0,0,0,0.0,0,0,0,38.7,38.7,2011
2,3,Calvin Johnson,DET,WR,15,OAK,14,9,64.3,214,...,23.8,0,0,0.0,0,0,0,37.9,37.9,2011
3,4,Torrey Smith,BAL,WR,3,LAR,8,5,62.5,152,...,30.4,1,10,10.0,0,0,0,36.7,36.7,2011
4,5,Miles Austin,DAL,WR,2,SF,15,9,60.0,143,...,15.9,1,-2,-2.0,0,1,0,36.6,36.6,2011


In [4]:
# Pull data from other workbooks
%store -r awayTable
%store -r homeTable
%store -r awayMatchup
%store -r homeMatchup
%store -r oddsTable
%store -r finalTeamGameDataTable

In [5]:
# Clean up the Team column to match the team column in awayTable and homeTable
teams1 = set(wr['Team'])
teams1.remove('SF')
teams2 = set(awayTable['team'])
teams2.remove('San Francisco 49ers')
t1List = list(teams1)
t2List = list(teams2)
t1List.sort()
t2List.sort()
zipTeams = zip(t1List, t2List)
teamDict = dict(list(zipTeams))
teamDict['SF'] = 'San Francisco 49ers'
teamDict

{'ARI': 'Arizona Cardinals',
 'ATL': 'Atlanta Falcons',
 'BAL': 'Baltimore Ravens',
 'BUF': 'Buffalo Bills',
 'CAR': 'Carolina Panthers',
 'CHI': 'Chicago Bears',
 'CIN': 'Cincinnati Bengals',
 'CLE': 'Cleveland Browns',
 'DAL': 'Dallas Cowboys',
 'DEN': 'Denver Broncos',
 'DET': 'Detroit Lions',
 'GB': 'Green Bay Packers',
 'HOU': 'Houston Texans',
 'IND': 'Indianapolis Colts',
 'JAX': 'Jacksonville Jaguars',
 'KC': 'Kansas City Chiefs',
 'LAC': 'Los Angeles Chargers',
 'LAR': 'Los Angeles Rams',
 'MIA': 'Miami Dolphins',
 'MIN': 'Minnesota Vikings',
 'NE': 'New England Patriots',
 'NO': 'New Orleans Saints',
 'NYG': 'New York Giants',
 'NYJ': 'New York Jets',
 'OAK': 'Oakland Raiders',
 'PHI': 'Philadelphia Eagles',
 'PIT': 'Pittsburgh Steelers',
 'SEA': 'Seattle Seahawks',
 'TB': 'Tampa Bay Buccaneers',
 'TEN': 'Tennessee Titans',
 'WAS': 'Washington Redskins',
 'SF': 'San Francisco 49ers'}

In [6]:
teamDict2 = {'ARI': 'Cardinals',
 'ATL': 'Falcons',
 'BAL': 'Ravens',
 'BUF': 'Bills',
 'CAR': 'Panthers',
 'CHI': 'Bears',
 'CIN': 'Bengals',
 'CLE': 'Browns',
 'DAL': 'Cowboys',
 'DEN': 'Broncos',
 'DET': 'Lions',
 'GB': 'Packers',
 'HOU': 'Texans',
 'IND': 'Colts',
 'JAX': 'Jaguars',
 'KC': 'Chiefs',
 'LAC': 'Chargers',
 'LAR': 'Rams',
 'MIA': 'Dolphins',
 'MIN': 'Vikings',
 'NE': 'Patriots',
 'NO': 'Saints',
 'NYG': 'Giants',
 'NYJ': 'Jets',
 'OAK': 'Raiders',
 'PHI': 'Eagles',
 'PIT': 'Steelers',
 'SEA': 'Seahawks',
 'TB': 'Buccaneers',
 'TEN': 'Titans',
 'WAS': 'Redskins',
 'SF': '49ers'}

In [7]:
teamDict3 = zip(teamDict.values(),teamDict2.values())
teamDict3 = dict(list(teamDict3))
teamDict3

{'Arizona Cardinals': 'Cardinals',
 'Atlanta Falcons': 'Falcons',
 'Baltimore Ravens': 'Ravens',
 'Buffalo Bills': 'Bills',
 'Carolina Panthers': 'Panthers',
 'Chicago Bears': 'Bears',
 'Cincinnati Bengals': 'Bengals',
 'Cleveland Browns': 'Browns',
 'Dallas Cowboys': 'Cowboys',
 'Denver Broncos': 'Broncos',
 'Detroit Lions': 'Lions',
 'Green Bay Packers': 'Packers',
 'Houston Texans': 'Texans',
 'Indianapolis Colts': 'Colts',
 'Jacksonville Jaguars': 'Jaguars',
 'Kansas City Chiefs': 'Chiefs',
 'Los Angeles Chargers': 'Chargers',
 'Los Angeles Rams': 'Rams',
 'Miami Dolphins': 'Dolphins',
 'Minnesota Vikings': 'Vikings',
 'New England Patriots': 'Patriots',
 'New Orleans Saints': 'Saints',
 'New York Giants': 'Giants',
 'New York Jets': 'Jets',
 'Oakland Raiders': 'Raiders',
 'Philadelphia Eagles': 'Eagles',
 'Pittsburgh Steelers': 'Steelers',
 'Seattle Seahawks': 'Seahawks',
 'Tampa Bay Buccaneers': 'Buccaneers',
 'Tennessee Titans': 'Titans',
 'Washington Redskins': 'Redskins',
 'Sa

In [8]:
# Map dictionary keys to new values so we have matching team/opponent names across the board
wr['Team'] = wr['Team'].map(teamDict2)
wr['Opponent'] = wr['Opponent'].map(teamDict2)

# qbSalaries['Team'] = qbSalaries['Team'].map(teamDict2)
# qbSalaries['Opponent'] = qbSalaries['Opponent'].map(teamDict2)


awayMatchup['team_x'] = awayMatchup['team_x'].map(teamDict3)
awayMatchup['team_y'] = awayMatchup['team_y'].map(teamDict3)
homeMatchup['team_x'] = homeMatchup['team_x'].map(teamDict3)
homeMatchup['team_y'] = homeMatchup['team_y'].map(teamDict3)

awayMatchup.head()

,gameId,date,season,week,team_x,homeAway,stadium,roof,surface,avgPf,...,defDvoa,ptsMatchup,offMatchup,ovrMatchup,passMatchup,pblkMatchup,runMatchup,offDvoaMatchup,totalDvoaMatchup,pf
0,201811110kan,2018-11-11,2018,10,Cardinals,away,Arrowhead Stadium,outdoors,grass,14.857143,...,0.076,37.857143,-10.285714,-13.857143,-10.600000,-16.071429,0.657143,-0.307,-0.764,14
1,201610300car,2016-10-30,2016,8,Cardinals,away,Bank of America Stadium,outdoors,grass,22.714286,...,0.031,51.285714,3.000000,4.457143,-2.228571,3.257143,-3.157143,-0.041,0.170,20
2,201612240sea,2016-12-24,2016,16,Cardinals,away,CenturyLink Field,outdoors,turf,25.857143,...,-0.128,43.857143,0.757143,-3.042857,7.171429,-14.242857,-9.314286,-0.225,-0.155,34
3,201812300sea,2018-12-30,2018,17,Cardinals,away,CenturyLink Field,outdoors,turf,13.000000,...,0.010,36.857143,-7.400000,-13.642857,-7.628571,-6.100000,-7.414286,-0.406,-0.520,24
4,201712310sea,2017-12-31,2017,17,Cardinals,away,CenturyLink Field,outdoors,turf,18.571429,...,-0.052,41.000000,-9.228571,-2.514286,-15.628571,-5.085714,-6.285714,-0.254,-0.144,26


In [9]:
#turn seasons to numeric instead of strings
wr['season'] = pd.to_numeric(wr['season'])
# qbSalaries['season'] = pd.to_numeric(qbSalaries['season'])

# Merge qb table with matchup tables, then put the two together
wrTable1 = pd.merge(wr,awayMatchup,how='inner',left_on=['Team','season','Week'],right_on=['team_x','season','week'])

wrTable2 = pd.merge(wr,homeMatchup,how='inner',left_on=['Team','season','Week'],right_on=['team_x','season','week'])

wrTableFinal = pd.concat([wrTable1,wrTable2],axis=0)

wrTableFinal.head()

# rbTableFinal = pd.merge(rbTable,rbSalaries,how='inner',left_on=['Name', 'Team', 'Position', 'Week', 'Opponent','season'],right_on=['Name', 'Team', 'Position', 'Week', 'Opponent','season'])
# rbTableFinal.columns

,Rank,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,...,defDvoa,ptsMatchup,offMatchup,ovrMatchup,passMatchup,pblkMatchup,runMatchup,offDvoaMatchup,totalDvoaMatchup,pf
0,1,Wes Welker,Patriots,WR,3,Bills,20,16,80.0,217,...,0.124,57.857143,18.985714,12.771429,11.371429,3.942857,10.485714,0.810,0.206,31
1,3,Calvin Johnson,Lions,WR,15,Raiders,14,9,64.3,214,...,0.072,56.142857,3.885714,-0.857143,-2.100000,10.700000,-1.471429,0.134,0.145,28
2,160,Nate Burleson,Lions,WR,15,Raiders,11,7,63.6,81,...,0.072,56.142857,3.885714,-0.857143,-2.100000,10.700000,-1.471429,0.134,0.145,28
3,463,Titus Young,Lions,WR,15,Raiders,10,5,50.0,21,...,0.072,56.142857,3.885714,-0.857143,-2.100000,10.700000,-1.471429,0.134,0.145,28
4,4,Torrey Smith,Ravens,WR,3,Rams,8,5,62.5,152,...,0.248,45.857143,10.585714,12.114286,9.542857,-5.900000,4.585714,0.223,0.661,37


In [10]:
wrs = wrTableFinal[(wrTableFinal['ReceivingTargets'] >= 1)]

In [11]:
wrs.head()

,Rank,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,...,defDvoa,ptsMatchup,offMatchup,ovrMatchup,passMatchup,pblkMatchup,runMatchup,offDvoaMatchup,totalDvoaMatchup,pf
0,1,Wes Welker,Patriots,WR,3,Bills,20,16,80.0,217,...,0.124,57.857143,18.985714,12.771429,11.371429,3.942857,10.485714,0.810,0.206,31
1,3,Calvin Johnson,Lions,WR,15,Raiders,14,9,64.3,214,...,0.072,56.142857,3.885714,-0.857143,-2.100000,10.700000,-1.471429,0.134,0.145,28
2,160,Nate Burleson,Lions,WR,15,Raiders,11,7,63.6,81,...,0.072,56.142857,3.885714,-0.857143,-2.100000,10.700000,-1.471429,0.134,0.145,28
3,463,Titus Young,Lions,WR,15,Raiders,10,5,50.0,21,...,0.072,56.142857,3.885714,-0.857143,-2.100000,10.700000,-1.471429,0.134,0.145,28
4,4,Torrey Smith,Ravens,WR,3,Rams,8,5,62.5,152,...,0.248,45.857143,10.585714,12.114286,9.542857,-5.900000,4.585714,0.223,0.661,37


In [12]:
wrs = pd.merge(wrs,oddsTable,how='inner',left_on=['Team','season','Week'],right_on=['team','season','week'])

In [13]:
wrs.loc[wrs.pf > wrs.pa, 'winLose'] = 'Win'
wrs.loc[wrs.pf < wrs.pa, 'winLose'] = 'Loss'
wrs.loc[wrs.pf == wrs.pa, 'winLose'] = 'Tie'

wrs.loc[wrs.totalScore > wrs.total, 'result'] = 'over'
wrs.loc[wrs.totalScore < wrs.total, 'result'] = 'under'
wrs.loc[wrs.totalScore == wrs.total, 'result'] = 'push'

In [14]:
wrs.columns

Index(['Rank', 'Name', 'Team', 'Position', 'Week', 'Opponent',
       'ReceivingTargets', 'Receptions', 'ReceptionPercentage',
       'ReceivingYards', 'ReceivingTouchdowns', 'ReceivingLong',
       'ReceivingYardsPerTarget', 'ReceivingYardsPerReception',
       'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       'RushingTouchdowns', 'Fumbles', 'FumblesLost',
       'FantasyPointsPerGameFanDuel', 'FantasyPointsFanDuel', 'season',
       'gameId', 'date_x', 'week_x', 'team_x', 'homeAway', 'stadium', 'roof',
       'surface', 'avgPf', 'avgOvr_x', 'avgOff', 'avgPass', 'avgPblk',
       'avgRecv', 'avgRun', 'avgRblk', 'totalDvoa_x', 'offDvoa', 'stDvoa',
       'overUnder', 'pa', 'totalScore', 'team_y', 'avgPa', 'avgOvr_y',
       'avgDef', 'avgRdef', 'avgPrsh', 'avgCov', 'totalDvoa_y', 'defDvoa',
       'ptsMatchup', 'offMatchup', 'ovrMatchup', 'passMatchup', 'pblkMatchup',
       'runMatchup', 'offDvoaMatchup', 'totalDvoaMatchup', 'pf', 'date_y',
       'week_y', 'team', '

In [15]:
wrVars = ['Name', 'Team', 'Position','season', 'Week', 'Opponent', 'homeAway', 'surface','ReceivingTargets', 'Receptions', 'ReceptionPercentage',
       'ReceivingYards', 'ReceivingTouchdowns','defDvoa','avgCov','avgPass','passMatchup','offMatchup','offDvoaMatchup','spread','total','impliedTotal', 'winLose','pa', 'result','FantasyPointsFanDuel']

In [16]:
newWrs = wrs[wrVars]
newWrs.tail()

,Name,Team,Position,season,Week,Opponent,homeAway,surface,ReceivingTargets,Receptions,...,passMatchup,offMatchup,offDvoaMatchup,spread,total,impliedTotal,winLose,pa,result,FantasyPointsFanDuel
7749,Robby Anderson,Jets,WR,2018,7,Vikings,home,turf,10,3,...,1.385714,-0.942857,-0.137,3.0,44.5,20.75,Loss,37,over,5.9
7750,Marquez Valdes-Scantling,Packers,WR,2018,17,Lions,home,grass,8,3,...,6.085714,0.757143,0.291,-7.5,44.5,26.00,Loss,31,under,5.8
7751,Josh Doctson,Redskins,WR,2018,7,Cowboys,home,grass,6,3,...,-7.657143,-6.971429,-0.082,1.0,40.5,19.75,Win,17,under,5.7
7752,Devin Funchess,Panthers,WR,2018,1,Cowboys,home,grass,5,3,...,-2.885714,-6.285714,-0.061,-2.5,42.0,22.25,Win,8,under,5.6
7753,Seth Roberts,Raiders,WR,2018,6,Seahawks,home,grass,6,5,...,-5.514286,-4.271429,-0.049,3.0,47.5,22.25,Loss,27,under,5.6


In [17]:
corr = newWrs.corr()
corr.style.background_gradient(cmap='coolwarm')

,season,Week,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,defDvoa,avgCov,avgPass,passMatchup,offMatchup,offDvoaMatchup,spread,total,impliedTotal,pa,FantasyPointsFanDuel
season,1,-0.00325826,-0.0387592,0.00890673,0.0718262,-0.024867,0.00910917,-0.0477,0.175711,0.0536846,-0.0668971,-0.230856,-0.0548022,-0.0182631,0.108687,0.0778724,0.0072118,-0.00131853
Week,-0.00325826,1,-0.00515061,-0.0199685,-0.0209284,-0.0180227,0.00135377,0.0059282,0.0467201,0.00915025,-0.0218754,-0.0442433,0.00847322,0.00194038,-0.01128,-0.00811871,-0.0334248,-0.00777578
ReceivingTargets,-0.0387592,-0.00515061,1,0.821909,-0.315046,0.577177,0.0190316,-0.033449,0.0247923,0.00794091,-0.00914685,-0.00106258,-0.0200547,0.0326672,0.0270927,-0.0110451,0.120105,0.471309
Receptions,0.00890673,-0.0199685,0.821909,1,0.212789,0.678205,0.0555326,-0.0259936,0.0165164,0.0625992,0.0396277,0.0344967,0.0289983,-0.0333232,0.0734353,0.0697858,0.0671663,0.584032
ReceptionPercentage,0.0718262,-0.0209284,-0.315046,0.212789,1,0.115509,0.0483453,0.00876463,-0.022229,0.0822321,0.0794864,0.0592871,0.0730098,-0.107306,0.0676434,0.126972,-0.0941024,0.132341
ReceivingYards,-0.024867,-0.0180227,0.577177,0.678205,0.115509,1,0.170255,-0.0136813,-0.00597713,0.0570504,0.0492475,0.058685,0.0336686,-0.0319091,0.0817914,0.0734667,0.0770139,0.77674
ReceivingTouchdowns,0.00910917,0.00135377,0.0190316,0.0555326,0.0483453,0.170255,1,0.0081999,-0.0328317,0.055535,0.0648085,0.0571217,0.0545843,-0.0686066,0.0675565,0.0952541,0.040255,0.709216
defDvoa,-0.0477,0.0059282,-0.033449,-0.0259936,0.00876463,-0.0136813,0.0081999,1,-0.456693,-0.0285577,0.26241,0.346176,0.604752,-0.150429,0.219697,0.250295,-0.00921939,-0.0104981
avgCov,0.175711,0.0467201,0.0247923,0.0165164,-0.022229,-0.00597713,-0.0328317,-0.456693,1,0.0265517,-0.603291,-0.599827,-0.28898,0.243215,-0.0996751,-0.256731,0.0571654,-0.0214552
avgPass,0.0536846,0.00915025,0.00794091,0.0625992,0.0822321,0.0570504,0.055535,-0.0285577,0.0265517,1,0.781221,0.525306,0.456882,-0.385165,0.374515,0.532014,-0.0401116,0.0766406


In [18]:
lookVars = ['Name', 'Team', 'season', 'Week', 'Opponent','homeAway',
       'ReceivingTargets', 'Receptions','ReceivingYards', 'ReceivingTouchdowns','spread', 'total','result',
       'impliedTotal','pa', 'FantasyPointsFanDuel']

In [19]:
panth = newWrs[(newWrs['Name'] == 'Julian Edelman') & (newWrs['season'] >= 2012)][lookVars]
# panth.corr().style.background_gradient(cmap='coolwarm')
panth[(panth['ReceivingTargets'] > 4) & (panth['Opponent'].isin(['Bills','Jets','Dolphins']))].sort_values('FantasyPointsFanDuel',ascending=False)

,Name,Team,season,Week,Opponent,homeAway,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,spread,total,result,impliedTotal,pa,FantasyPointsFanDuel
1959,Julian Edelman,Patriots,2015,2,Bills,away,19,11,97,2,2.0,45.0,over,21.50,32,28.4
2406,Julian Edelman,Patriots,2016,17,Dolphins,away,13,8,151,1,-8.0,47.0,over,27.50,14,27.7
999,Julian Edelman,Patriots,2013,15,Dolphins,away,19,13,139,1,2.5,46.5,under,22.00,24,26.4
5882,Julian Edelman,Patriots,2015,8,Dolphins,home,9,7,81,2,-8.0,51.0,under,29.50,7,23.6
1038,Julian Edelman,Patriots,2013,1,Bills,away,9,7,79,2,-10.0,50.5,under,30.25,21,23.4
3481,Julian Edelman,Patriots,2018,14,Dolphins,away,12,9,86,1,-9.5,49.5,over,29.50,34,19.1
5451,Julian Edelman,Patriots,2014,15,Dolphins,home,10,7,88,1,-9.0,49.0,over,29.00,13,18.9
7465,Julian Edelman,Patriots,2018,17,Jets,home,6,5,69,1,-13.5,46.5,under,30.00,3,17.8
3556,Julian Edelman,Patriots,2018,12,Jets,away,5,4,84,1,-12.5,47.0,under,29.75,13,16.4
3573,Julian Edelman,Patriots,2018,8,Bills,away,10,9,104,0,-14.0,44.0,under,29.00,6,16.2


In [20]:
print(np.mean(panth[(panth['ReceivingTargets'] > 4) & (panth['Opponent'].isin(['Bills','Jets','Dolphins']))]['FantasyPointsFanDuel']))
print(np.mean(panth[(panth['ReceivingTargets'] > 4) & (np.logical_not(panth['Opponent'].isin(['Bills','Jets','Dolphins'])))]['FantasyPointsFanDuel']))

16.331818181818182
13.105365853658535


In [30]:
panth = newWrs[(newWrs['Team'] == 'Cowboys') & (newWrs['season'] >= 2015) & (newWrs['ReceivingTargets'] >= 8) & (newWrs['result'] == 'over')][lookVars]
# panth.corr().style.background_gradient(cmap='coolwarm')
panth[lookVars].sort_values('FantasyPointsFanDuel',ascending=False)

,Name,Team,season,Week,Opponent,homeAway,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,spread,total,result,impliedTotal,pa,FantasyPointsFanDuel
7283,Amari Cooper,Cowboys,2018,14,Eagles,home,13,10,217,3,-3.0,45.5,over,24.25,23,44.7
7287,Amari Cooper,Cowboys,2018,12,Redskins,home,9,8,180,2,-7.0,40.0,over,23.50,23,34.0
5849,Cole Beasley,Cowboys,2015,9,Eagles,home,11,9,112,2,3.0,44.0,over,20.50,33,27.7
7323,Cole Beasley,Cowboys,2018,6,Jaguars,home,11,9,101,2,3.0,40.0,over,18.50,7,26.6
2475,Dez Bryant,Cowboys,2016,10,Steelers,away,9,6,116,1,3.0,50.5,over,23.75,30,20.6
6407,Dez Bryant,Cowboys,2016,8,Eagles,home,14,4,113,1,-5.0,44.0,over,24.50,23,19.3
5850,Dez Bryant,Cowboys,2015,9,Eagles,home,8,5,104,1,3.0,44.0,over,20.50,33,18.9
3050,Dez Bryant,Cowboys,2017,7,49ers,away,10,7,63,1,-6.0,48.0,over,27.00,10,15.8
3056,Dez Bryant,Cowboys,2017,2,Broncos,away,16,7,59,1,-2.5,44.0,over,23.25,42,15.4
7560,Amari Cooper,Cowboys,2018,9,Titans,home,8,5,58,1,-4.0,40.0,over,22.00,28,14.3


In [102]:
th = newWrs[(newWrs['Name'] == 'Will Fuller') & (newWrs['season'] >= 2015)]
# th[wrVars].corr().style.background_gradient(cmap='coolwarm')

In [103]:
th[lookVars].sort_values('FantasyPointsFanDuel',ascending=False)

,Name,Team,season,Week,Opponent,homeAway,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,spread,total,result,impliedTotal,pa,FantasyPointsFanDuel
2876,Will Fuller,Texans,2017,8,Seahawks,away,8,5,125,2,6.0,45.0,over,19.50,41,27.0
6339,Will Fuller,Texans,2016,4,Titans,home,9,7,81,1,-4.0,40.0,over,22.00,20,23.6
3461,Will Fuller,Texans,2018,2,Titans,away,9,8,113,1,-3.5,41.0,under,22.25,20,21.3
7362,Will Fuller,Texans,2018,8,Dolphins,home,6,5,124,1,-7.5,45.0,over,26.25,23,20.9
6409,Will Fuller,Texans,2016,1,Bears,home,11,5,107,1,-6.0,42.5,under,24.25,14,19.2
6825,Will Fuller,Texans,2017,5,Chiefs,home,3,2,57,2,1.5,45.5,over,22.00,42,19.2
7446,Will Fuller,Texans,2018,3,Giants,home,11,5,101,1,-6.5,44.5,over,25.50,27,18.6
6850,Will Fuller,Texans,2017,4,Titans,home,6,4,35,2,2.5,43.5,over,20.50,14,17.9
7057,Will Fuller,Texans,2017,6,Browns,home,5,2,62,1,-7.0,46.0,over,26.50,17,13.2
3392,Will Fuller,Texans,2018,4,Colts,away,5,4,49,1,-1.0,48.5,over,24.75,34,12.9


In [47]:
class WR:
    
    def __init__(self,name,table):
        self.name = name
        self.table = table[table['Name'] == name]
        
    def get_table(self):
        return self.table
    
    def corrMatrix(self,features):
        correlations = self.table[features].corr()
        return correlations.style.background_gradient(cmap='coolwarm')
    
    def scatter(self,var1,var2):
        plt.scatter(self.table[var1],self.table[var2])
        plt.title(self.name)
        plt.xlabel(var1)
        plt.ylabel(var2)
        plt.show()
        
    def impliedTot(self, value):
        return self.table[self.table['impliedTotal'] >= value][wrVars]

In [48]:
ka = WR('Julio Jones',newWrs)
ka.corrMatrix(wrVars)

,season,Week,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,defDvoa,avgCov,avgPass,passMatchup,offMatchup,offDvoaMatchup,spread,total,impliedTotal,FantasyPointsFanDuel
season,1,-0.00191025,0.157167,0.148875,-0.0695267,0.112653,-0.150591,-0.0750559,0.18085,0.105007,-0.0901927,-0.103349,-0.0672435,0.00141104,0.288441,0.223014,0.00511734
Week,-0.00191025,1,-0.0908816,-0.156021,-0.126184,-0.0856558,0.0844819,0.112585,0.111264,-0.0137742,-0.100017,-0.0839412,0.0232396,-0.00954431,0.0396993,0.0396376,-0.0365227
ReceivingTargets,0.157167,-0.0908816,1,0.815398,-0.291135,0.560989,0.0300959,-0.176136,0.145924,-0.0137794,-0.128738,-0.0877522,-0.0924595,0.176725,0.0895229,-0.0926744,0.458388
Receptions,0.148875,-0.156021,0.815398,1,0.284005,0.751338,0.202675,-0.0923144,0.0983044,-0.0811339,-0.127547,-0.0614138,-0.0722075,0.232817,0.161912,-0.087891,0.674311
ReceptionPercentage,-0.0695267,-0.126184,-0.291135,0.284005,1,0.300221,0.320861,0.142494,-0.0691104,-0.13953,-0.0220056,0.0191891,0.00949425,0.112678,0.0624027,-0.0549476,0.367524
ReceivingYards,0.112653,-0.0856558,0.560989,0.751338,0.300221,1,0.300671,-0.126633,0.20257,-0.0345598,-0.18748,-0.16469,-0.0880395,0.235659,0.0954742,-0.142167,0.852365
ReceivingTouchdowns,-0.150591,0.0844819,0.0300959,0.202675,0.320861,0.300671,1,0.0927024,0.0511392,-0.1758,-0.142248,-0.0463106,0.0642348,0.0204394,-0.0702337,-0.0733883,0.721911
defDvoa,-0.0750559,0.112585,-0.176136,-0.0923144,0.142494,-0.126633,0.0927024,1,-0.448059,-0.230764,0.240154,0.254064,0.668837,-0.192004,0.286548,0.399161,-0.044926
avgCov,0.18085,0.111264,0.145924,0.0983044,-0.0691104,0.20257,0.0511392,-0.448059,1,0.00988804,-0.822967,-0.734265,-0.385381,0.295472,-0.147323,-0.38591,0.143515
avgPass,0.105007,-0.0137742,-0.0137794,-0.0811339,-0.13953,-0.0345598,-0.1758,-0.230764,0.00988804,1,0.559924,0.385939,0.178821,-0.172374,0.140829,0.267815,-0.114227


In [54]:
allen = ka.get_table()
allen[(allen['surface'] == 'turf')]

,Name,Team,Position,season,Week,Opponent,homeAway,surface,ReceivingTargets,Receptions,...,avgPass,passMatchup,offMatchup,offDvoaMatchup,spread,total,impliedTotal,winLose,result,FantasyPointsFanDuel
29,Julio Jones,Falcons,WR,2011,9,Colts,away,turf,4,3,...,66.514286,11.285714,7.071429,0.324,-6.5,45.0,25.75,Win,under,29.9
92,Julio Jones,Falcons,WR,2011,16,Saints,away,turf,14,8,...,73.557143,18.757143,18.942857,0.271,7.0,52.5,22.75,Loss,over,20.8
149,Julio Jones,Falcons,WR,2011,4,Seahawks,away,turf,17,11,...,68.414286,9.528571,-1.257143,0.009,-6.0,39.0,22.50,Win,over,18.2
479,Julio Jones,Falcons,WR,2012,16,Lions,away,turf,11,7,...,68.357143,4.542857,8.200000,0.132,-3.5,50.0,26.75,Win,under,15.9
729,Julio Jones,Falcons,WR,2012,10,Saints,away,turf,5,4,...,76.385714,22.814286,19.471429,0.312,-2.0,53.5,27.75,Loss,over,9.5
1210,Julio Jones,Falcons,WR,2013,1,Saints,away,turf,9,7,...,75.014286,14.142857,17.457143,0.140,3.5,54.5,25.50,Loss,under,15.1
1539,Julio Jones,Falcons,WR,2014,7,Ravens,away,turf,8,5,...,74.385714,16.200000,6.028571,0.071,6.5,50.0,21.75,Loss,under,8.1
1605,Julio Jones,Falcons,WR,2014,2,Bengals,away,turf,13,7,...,70.142857,3.485714,3.557143,0.356,5.5,49.0,21.75,Loss,under,18.3
1656,Julio Jones,Falcons,WR,2014,5,Giants,away,turf,16,11,...,71.700000,2.957143,0.800000,0.155,4.5,50.5,23.00,Loss,under,16.0
1666,Julio Jones,Falcons,WR,2014,4,Vikings,away,turf,8,6,...,71.471429,16.714286,8.000000,0.268,-5.0,47.5,26.25,Loss,over,11.2


In [63]:
np.mean(allen[(allen['homeAway'] == 'away')]['FantasyPointsFanDuel'])

17.11132075471698